# SVM Fake News Classifier - Classificação de Notícias Externas

Este notebook utiliza o modelo SVM treinado para classificar notícias da base externa `party_news.parquet`.

## Objetivo
Carregar o modelo SVM já treinado e aplicá-lo na base de dados externa para detectar possíveis fake news.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Carregar Modelo e Vetorizador

In [ ]:
# Carregar o modelo SVM treinado
with open('../models/svm_fake_news_classifier.pkl', 'rb') as f:
    svm_model = pickle.load(f)

# Carregar o vetorizador TF-IDF
with open('../models/tfidf_vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)

print("Modelo e vetorizador carregados com sucesso!")
print(f"Tipo do modelo: {type(svm_model)}")
print(f"Número de features do TF-IDF: {len(tfidf.get_feature_names_out())}")

## 2. Carregar Dados Externos (Party News)

In [ ]:
# Carregar dados externos
df_external = pd.read_parquet('../data/external/party_news.parquet')

print(f"Shape dos dados externos: {df_external.shape}")
print(f"\nColunas disponíveis: {df_external.columns.tolist()}")
print(f"\nPrimeiras linhas:")
df_external.head()

In [ ]:
# Verificar informações sobre os dados
print("Informações sobre os dados:")
df_external.info()

print("\nEstatísticas descritivas:")
df_external.describe()

## 3. Preparar Dados para Classificação

In [ ]:
# Verificar se existe coluna de texto pré-processado
# Ajustar o nome da coluna conforme necessário
text_column = None
for col in ['preprocessed_text', 'text', 'content', 'full_text']:
    if col in df_external.columns:
        text_column = col
        break

if text_column is None:
    raise ValueError("Não foi encontrada coluna de texto nos dados!")

print(f"Usando coluna: {text_column}")

# Remover valores nulos
df_clean = df_external[df_external[text_column].notna() & (df_external[text_column] != '')].copy()
print(f"\nDados após limpeza: {df_clean.shape}")
print(f"Registros removidos: {len(df_external) - len(df_clean)}")

## 4. Vetorizar Textos

In [ ]:
# Vetorizar os textos usando o TF-IDF já treinado
X_external = tfidf.transform(df_clean[text_column].values)

print(f"Shape da matriz TF-IDF: {X_external.shape}")
print(f"Número de documentos: {X_external.shape[0]}")
print(f"Número de features: {X_external.shape[1]}")

## 5. Fazer Predições

In [ ]:
# Fazer predições
predictions = svm_model.predict(X_external)

# Adicionar predições ao dataframe
df_clean['is_fake_prediction'] = predictions

# Contar predições
print("Distribuição das predições:")
print(df_clean['is_fake_prediction'].value_counts())
print("\nPorcentagem:")
print(df_clean['is_fake_prediction'].value_counts(normalize=True) * 100)

## 6. Análise dos Resultados

In [ ]:
# Visualizar distribuição das predições
plt.figure(figsize=(10, 6))
df_clean['is_fake_prediction'].value_counts().plot(kind='bar', color=['green', 'red'])
plt.title('Distribuição das Predições - Party News')
plt.xlabel('Classificação (0=True, 1=Fake)')
plt.ylabel('Quantidade')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Se existir coluna de partido, analisar por partido
party_column = None
for col in ['party', 'partido', 'source', 'fonte']:
    if col in df_clean.columns:
        party_column = col
        break

if party_column:
    print(f"\nAnálise por {party_column}:")
    party_analysis = df_clean.groupby(party_column)['is_fake_prediction'].agg(['count', 'sum', 'mean'])
    party_analysis.columns = ['Total', 'Fake_Count', 'Fake_Percentage']
    party_analysis['Fake_Percentage'] = party_analysis['Fake_Percentage'] * 100
    party_analysis = party_analysis.sort_values('Fake_Percentage', ascending=False)
    print(party_analysis)
    
    # Visualizar
    plt.figure(figsize=(12, 6))
    party_analysis['Fake_Percentage'].plot(kind='bar')
    plt.title('Porcentagem de Fake News por Partido/Fonte')
    plt.xlabel(party_column.capitalize())
    plt.ylabel('% Fake News')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("Coluna de partido/fonte não encontrada")

## 7. Exemplos de Notícias Classificadas como Fake

In [ ]:
# Mostrar exemplos de notícias classificadas como fake
fake_news = df_clean[df_clean['is_fake_prediction'] == 1]

print(f"Total de notícias classificadas como FAKE: {len(fake_news)}")
print("\nExemplos de notícias classificadas como FAKE:\n")

for idx, row in fake_news.head(10).iterrows():
    print(f"{'='*80}")
    if party_column and party_column in row:
        print(f"Partido/Fonte: {row[party_column]}")
    print(f"Texto: {row[text_column][:200]}...")
    print()

## 8. Salvar Resultados

In [ ]:
# Salvar resultados em um novo arquivo
output_path = '../data/processed/party_news_classified.parquet'
df_clean.to_parquet(output_path, index=False)
print(f"Resultados salvos em: {output_path}")

# Salvar também um CSV com apenas as fake news
fake_news_path = '../data/processed/party_news_fake_only.csv'
fake_news.to_csv(fake_news_path, index=False)
print(f"Fake news salvas em: {fake_news_path}")

## 9. Resumo Final

In [ ]:
print("="*80)
print("RESUMO DA CLASSIFICAÇÃO")
print("="*80)
print(f"Total de notícias analisadas: {len(df_clean)}")
print(f"Notícias classificadas como TRUE: {(df_clean['is_fake_prediction'] == 0).sum()}")
print(f"Notícias classificadas como FAKE: {(df_clean['is_fake_prediction'] == 1).sum()}")
print(f"\nPorcentagem de FAKE news: {(df_clean['is_fake_prediction'].mean() * 100):.2f}%")
print(f"Porcentagem de TRUE news: {((1 - df_clean['is_fake_prediction'].mean()) * 100):.2f}%")
print("="*80)